In [1]:
import numpy as np
import torch
import transformers

from life_after_bert import MCDataset, evaluate_encoder

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "roberta-large"
task_name = "Age Comparison"
num_choices = 2

In [3]:
model = transformers.AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
dataset = MCDataset.load_data(task_name, num_choices, tokenizer)

2022-04-21 11:38:10 | INFO | data.py | Loading jsonl file from /home/kzhao/life-after-bert/tests/data/oLMpics_age_comparison_dev.jsonl


In [4]:
accuracy, (all_answers, all_preds) = evaluate_encoder(model, tokenizer, dataset, device=device)
accuracy

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

0.986